In [2]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 18.28 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
import pandas as pd

train_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')
train_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [4]:
test_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')
test_data.head()
#train_data.shape

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [5]:
sample_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/sample_submission.csv')
sample_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,0
2,2082.jpg,0
3,808.jpg,0
4,1907.jpg,0


In [6]:
import zipfile
images_dir = '/content/Qualcomm-DL-Hackathon/train/images'
with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-1.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-2.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

In [7]:
#Move all images to single folder

import shutil
import os

source_dir1 = '/content/Qualcomm-DL-Hackathon/train/images/images part-1'
source_dir2 = '/content/Qualcomm-DL-Hackathon/train/images/images part-2'

destination_dir = '/content/Qualcomm-DL-Hackathon/dataset/images'
os.makedirs(destination_dir, exist_ok=True)

def copy_files(source, destination):
    for filename in os.listdir(source):
        source_file = os.path.join(source, filename)
        destination_file = os.path.join(destination, filename)
        if os.path.isfile(source_file):
          shutil.copy(source_file, destination_dir)

copy_files(source_dir1, destination_dir)
copy_files(source_dir2, destination_dir)

shutil.rmtree(source_dir1)
shutil.rmtree(source_dir2)

print(f'Images pat: {destination_dir}')

Images pat: /content/Qualcomm-DL-Hackathon/dataset/images


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torch.utils.data import random_split

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from PIL import Image
class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)

        # Check if there's a second column for labels
        if self.data.shape[1] > 1:
            label = self.data.iloc[idx, 1]
        else:
            label = 0  # or None, depending on how you want to handle it

        if self.transform:
            image = self.transform(image)
        return image, label

In [10]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet mean/std
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
train_dataset = ImageDataset(csv_file='/content/Qualcomm-DL-Hackathon/train/train.csv',
                             root_dir='/content/Qualcomm-DL-Hackathon/dataset/images',
                             transform=train_transforms)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

In [12]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 32 * 32, 128) # Adjust input size based on image dimensions and pooling
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 2)  # 2 output classes

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32) # Adjust input size based on image dimensions and pooling
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop (not for lenet)
num_epochs = 100 # Adjust as needed
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
    if loss.item() < 0.0001:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.5f}, Epoch_Loss: {epoch_loss:.5f}")
            print("Loss is less than 0.001. Breaking the loop.")
            break

    if loss.item() < 0.0001:
        break
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.5f}, Epoch_Loss: {epoch_loss:.5f}")

print("Training finished.")

In [15]:
# Evaluate the model on the validation set
model.eval()  # Set the model to evaluation mode
val_correct = 0
val_total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

val_accuracy = 100 * val_correct / val_total
print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Accuracy: 82.12%


In [16]:
# Hyperparameters to tune
learning_rates = [0.001]
batch_sizes = [64, 128, 256]
dropout_rates = [0.3, 0.5]
hidden_layers = [3]  # 1 hidden layer or 2 hidden layers
hidden_units = [128, 256]  # Size of each hidden layer
num_epochs = 100

# Placeholder for the best model and hyperparameters
best_model = None
best_accuracy = 0
best_params = {}

# Grid search loop over hyperparameters
for lr in learning_rates:
    for batch_size in batch_sizes:
        for dropout_rate in dropout_rates:
            for num_hidden_layers in hidden_layers:
                for hidden_units_per_layer in hidden_units:

                    # Recreate data loaders with the current batch size
                    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

                    # Define the CNN model architecture with the current number of hidden layers and dropout rate
                    conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
                    conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
                    conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
                    pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
                    fc_layers = []

                    # Create hidden layers based on the number of hidden layers
                    input_size = 128 * 16 * 16  # Image size after convolution and pooling
                    for i in range(num_hidden_layers):
                        fc_layers.append(nn.Linear(input_size, hidden_units_per_layer))
                        fc_layers.append(nn.ReLU())
                        fc_layers.append(nn.Dropout(dropout_rate))
                        input_size = hidden_units_per_layer  # Update input size for the next layer

                    fc_layers.append(nn.Linear(input_size, 5))  # Output layer for 5 classes (cycling, dancing, drinking, eating, sitting)

                    # Combine layers into a model
                    model = nn.Sequential(
                        conv1,
                        nn.ReLU(),
                        pool,
                        conv2,
                        nn.ReLU(),
                        pool,
                        conv3,
                        nn.ReLU(),
                        pool,
                        nn.Flatten(),
                        *fc_layers
                    ).to(device)

                    # Define the loss function and optimizer for this combination of hyperparameters
                    criterion = nn.CrossEntropyLoss()
                    optimizer = optim.Adam(model.parameters(), lr=lr)

                    # Train the model
                    for epoch in range(num_epochs):
                        model.train()
                        running_loss = 0.0
                        correct = 0
                        total = 0

                        for inputs, labels in train_loader:
                            inputs, labels = inputs.to(device), labels.to(device)

                            optimizer.zero_grad()
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                            loss.backward()
                            optimizer.step()

                            running_loss += loss.item()
                            _, predicted = torch.max(outputs, 1)
                            total += labels.size(0)
                            correct += (predicted == labels).sum().item()

                        epoch_loss = running_loss / len(train_loader)
                        epoch_accuracy = 100 * correct / total
                        print(f"Epoch [{epoch+1}/{num_epochs}] for lr={lr}, batch_size={batch_size}, dropout={dropout_rate}, hidden_layers={num_hidden_layers}, hidden_units={hidden_units_per_layer}: Loss={epoch_loss:.4f}, Accuracy={epoch_accuracy:.2f}%")
                        if epoch_loss < 0.001:
                          print(f"Epoch [{epoch+1}/{num_epochs}] for lr={lr}, batch_size={batch_size}, dropout={dropout_rate}, hidden_layers={num_hidden_layers}, hidden_units={hidden_units_per_layer}: Loss={epoch_loss:.4f}, Accuracy={epoch_accuracy:.2f}%")
                          print("Loss is less than 0.001. Breaking the loop.")
                          break

                        if epoch_loss < 0.001:
                          break

                    # Evaluate on the validation set after training
                    model.eval()
                    val_correct = 0
                    val_total = 0
                    with torch.no_grad():
                        for inputs, labels in val_loader:
                            inputs, labels = inputs.to(device), labels.to(device)
                            outputs = model(inputs)
                            _, predicted = torch.max(outputs, 1)
                            val_total += labels.size(0)
                            val_correct += (predicted == labels).sum().item()

                    val_accuracy = 100 * val_correct / val_total
                    print(f"Validation Accuracy for lr={lr}, batch_size={batch_size}, dropout={dropout_rate}, hidden_layers={num_hidden_layers}, hidden_units={hidden_units_per_layer}: {val_accuracy:.2f}%")


Epoch [1/100] for lr=0.001, batch_size=64, dropout=0.3, hidden_layers=3, hidden_units=128: Loss=0.9234, Accuracy=47.57%
Epoch [2/100] for lr=0.001, batch_size=64, dropout=0.3, hidden_layers=3, hidden_units=128: Loss=0.6331, Accuracy=64.21%
Epoch [3/100] for lr=0.001, batch_size=64, dropout=0.3, hidden_layers=3, hidden_units=128: Loss=0.5529, Accuracy=75.30%
Epoch [4/100] for lr=0.001, batch_size=64, dropout=0.3, hidden_layers=3, hidden_units=128: Loss=0.4686, Accuracy=79.26%
Epoch [5/100] for lr=0.001, batch_size=64, dropout=0.3, hidden_layers=3, hidden_units=128: Loss=0.4359, Accuracy=79.86%
Epoch [6/100] for lr=0.001, batch_size=64, dropout=0.3, hidden_layers=3, hidden_units=128: Loss=0.4120, Accuracy=82.83%
Epoch [7/100] for lr=0.001, batch_size=64, dropout=0.3, hidden_layers=3, hidden_units=128: Loss=0.3542, Accuracy=85.41%
Epoch [8/100] for lr=0.001, batch_size=64, dropout=0.3, hidden_layers=3, hidden_units=128: Loss=0.2622, Accuracy=89.51%
Epoch [9/100] for lr=0.001, batch_size=6

KeyboardInterrupt: 